<a href="https://colab.research.google.com/github/Vraddhi/soilClassification_annam/blob/main/Soil_Classification_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install numpy pandas matplotlib scikit-learn opencv-python torch torchvision torchaudio


  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/soil_classification-2025/train_labels.csv')

label_mapping = {
    'Alluvial soil': 0,
    'Black Soil': 1,
    'Clay soil': 2,
    'Red soil': 3
}

df['label_enc'] = df['soil_type'].map(label_mapping)

print(df.head())


           image_id      soil_type  label_enc
0  img_ed005410.jpg  Alluvial soil          0
1  img_0c5ecd2a.jpg  Alluvial soil          0
2  img_ed713bb5.jpg  Alluvial soil          0
3  img_12c58874.jpg  Alluvial soil          0
4  img_eff357af.jpg  Alluvial soil          0


In [ ]:
import pandas as pd
import os
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
from tqdm import tqdm


In [ ]:
class SoilDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx]['image_id']
        label = int(self.dataframe.iloc[idx]['label_enc'])
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label_enc'], random_state=42)


In [ ]:
train_dataset = SoilDataset(train_df, '/content/drive/MyDrive/soil_classification-2025/train', transform=transform)
val_dataset = SoilDataset(val_df, '/content/drive/MyDrive/soil_classification-2025/train', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [ ]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 4)  # 4 soil classes

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 168MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(10):
    model.train()
    running_loss = 0.0

    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


100%|██████████| 31/31 [15:05<00:00, 29.19s/it]


Epoch 1, Loss: 0.3880461317396933


100%|██████████| 31/31 [04:31<00:00,  8.77s/it]


Epoch 2, Loss: 0.06047936826343498


100%|██████████| 31/31 [04:28<00:00,  8.67s/it]


...

Epoch 8, Loss: 0.00773362320427212


100%|██████████| 31/31 [04:25<00:00,  8.56s/it]


Epoch 9, Loss: 0.006139223180305693


100%|██████████| 31/31 [04:28<00:00,  8.65s/it]

Epoch 10, Loss: 0.002738499529509535


In [ ]:
# TEST
class TestSoilDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_name = self.dataframe.iloc[idx]['image_id']
        image_path = os.path.join(self.image_dir, image_name)
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image_name, image


In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/soil_classification-2025/test_ids.csv')


In [ ]:
test_dataset = TestSoilDataset(test_df, '/content/drive/MyDrive/soil_classification-2025/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
label_reverse_mapping = {0: 'Alluvial soil', 1: 'Black Soil', 2: 'Clay soil', 3: 'Red soil'}


In [ ]:
model.eval()
predictions = []

with torch.no_grad():
    for image_names, images in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predicted = predicted.cpu().numpy()

        for name, pred in zip(image_names, predicted):
            predictions.append((name, label_reverse_mapping[pred]))


In [ ]:
submission_df = pd.DataFrame(predictions, columns=['image_id', 'soil_type'])
submission_df.to_csv('submission.csv', index=False)


In [ ]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

f1s = f1_score(all_labels, all_preds, average=None)
print(f"Per-class F1 scores: {f1s}")
print(f"Min F1 score: {min(f1s)}")


Per-class F1 scores: [0.98076923 0.9787234  0.97560976 1.        ]
Min F1 score: 0.975609756097561
